In [1]:
import requests
import pandas as pd
import re
import time
import random
from bs4 import BeautifulSoup
import progressbar
import webbrowser
from tempfile import NamedTemporaryFile

In [2]:
# Criando um dicionário com os dados da requisição

datareq = {
	"conversationId": "",
	"dados.buscaInteiroTeor": "",
	"dados.pesquisarComSinonimos": [
		"S",
		"S"
	],
	"dados.buscaEmenta": "",
	"dados.nuProcOrigem": "",
	"dados.nuRegistro": "",
	"agenteSelectedEntitiesList": "",
	"contadoragente": "0",
	"contadorMaioragente": "0",
	"codigoCr": "",
	"codigoTr": "",
	"nmAgente": "",
	"juizProlatorSelectedEntitiesList": "",
	"contadorjuizProlator": "0",
	"contadorMaiorjuizProlator": "0",
	"codigoJuizCr": "",
	"codigoJuizTr": "",
	"nmJuiz": "",
	"classesTreeSelection.values": "",
	"classesTreeSelection.text": "",
    "assuntosTreeSelection.values": '',
	"comarcaSelectedEntitiesList": "",
	"contadorcomarca": "0",
	"contadorMaiorcomarca": "0",
	"cdComarca": "",
	"nmComarca": "",
	"secoesTreeSelection.values": "",
	"secoesTreeSelection.text": "",
	"dados.dtJulgamentoInicio": "",
	"dados.dtJulgamentoFim": "",
	"dados.dtPublicacaoInicio": "",
	"dados.dtPublicacaoFim": "",
	"dados.origensSelecionadas": "T",
	"tipoDecisaoSelecionados": [
        "A",
        "H",
        "D"
	],
	"dados.ordenarPor": "relevancia"
}

In [7]:
# Lista contendo os nomes e códigos dos assuntos pesquisados

assuntos = [
    ['Direito Tributário', '14,5913,5914,10527,10528,10529,10530,5915,10540,5916,5917,5933,5941,5944,5945,5946,5947,10531,5948,5950,5951,5952,5953,5954,5955,10690,5956,5957,5958,5959,5960,5961,5962,5963,5964,5965,5966,5967,5968,5969,5970,5971,5972,10534,10535,10536,10537,10538,5973,5974,5975,5976,5977,5978,5979,5980,5981,5982,5983,5984,5985,10539,5986,5987,5988,5989,10543,5990,5991,5992,10544,10545,10546,10549,10547,10550,10548,5993,10551,5994,5995,14950,5996,5997,5998,5999,6000,6001,6002,6003,6004,6005,6006,6007,6008,10556,10557,10558,10559,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,10560,6030,6031,6032,6033,6034,6035,6036,10563,6037,6038,6039,6040,6041,6042,6043,10561,10562,6044,6045,6046,6047,10565,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,6075,6076,6077,6078,6079,6080,6081,6082,6083,6084,6085,6086,6087,6088,6089,6090,6091,6092,6093'],
    ['Direito Previdenciário', '195,6094,6095,6096,6097,6098,6099,6100,6101,6102,6103,6104,6105,6106,6107,6108,6109,6110,6111,6112,6113,6115,6116,6117,6118,7757,10567,6119,6120,6121,6122,6123,6124,6125,6126,6127,11940,6128,6129,6130,6131,6132,6133,6134,6135,6136,6137,11941,11942,11943,11944,6138,6139,6140,6141,6142,6143,6144,6145,6146,6147,6148,6149,11945,6150,6151,6152,6153,6154,6155,6156,6157,6158,6159,6160,6161,6162,6164,6165,6166,6167,6168,6169,6170,6171,6172,6173,6174,6175,6176,6177,6178,6179,6181,6182,6183,6184,6185,6186,6187,6188,6189,6190'],
    ['Direito Penal' , '3463,3472,3491,3520,3547,5872,5874,3692,5555,5865,5873,5875,10620,10949,10950,10951,10952,11414,12217,12333,12342,15169,15174'],
    ['Direito do Trabalho' , '864,2567,1855,2569,8808,8809'],
    ['Direito Civil' , '899,10432,7947,7681,9981,9616,5626,5754,7673,10431,12935,12937,14205,15219,50297']
]

In [8]:
# Criando dataframe a ser preenchido

df = pd.DataFrame(columns=['Assunto', 'Texto original', 'Texto semi-tratado'])

In [33]:
# Realizando teste via browser

# Editando os assuntos no dicionário de requisição
datareq["assuntosTreeSelection.values"] = assuntos[0][1]
    
# Fazendo a requisição post para obter os cookies
with requests.post(
        url = "https://cjo.tjsp.jus.br/cjsg/resultadoCompleta.do",
        data = datareq
    ) as post_req:
    with NamedTemporaryFile("wb", delete=False, suffix=".html") as file:
        file.write(post_req.content)
    # open temporary file in a new browser tab as a web page
    webbrowser.open_new_tab(f"file://{file.name}")


In [10]:
# Definindo número de páginas a ser percorrida para cada assunto
n_pages = 1000
tot_int = len(assuntos)*n_pages

# Configurando a barra de progresso
widgets = [' [',
         progressbar.Timer(),
         '] ',
           progressbar.Bar('*'),' (',
           progressbar.ETA(), ') ',
          ]
bar = progressbar.ProgressBar(maxval = tot_int, widgets=widgets).start()

# Loop que realiza as requisições
i = 0
for linha in assuntos:

    # Editando os assuntos no dicionário de requisição
    datareq["assuntosTreeSelection.values"] = linha[1]
    
    # Fazendo a requisição post para obter os cookies
    with requests.post(
            url = "https://cjo.tjsp.jus.br/cjsg/resultadoCompleta.do",
            data = datareq
        ) as post_req:

        cookies = post_req.cookies


    time.sleep(2+ random.uniform(0, 3))

    # Percorrendo 1000 páginas
    for page in range(1, n_pages+1):

        with requests.get(
                                url = "https://cjo.tjsp.jus.br/cjsg/trocaDePagina.do?tipoDeDecisao=A&pagina="+str(page),
                                cookies=cookies
                                ) as r:
            pagehtml = BeautifulSoup(
                                    r.content,
                                    'html.parser'
            )

        # Realizando uma limpeza inicial do texto
        for elem in pagehtml.find_all(string=re.compile("Ementa:")):
            textappend = re.sub('\s\s+', ' ',
                         re.sub(' +', '', 
                         re.sub('Ementa:', '', elem.find_parent().find_parent().text).replace('\r', '').replace('\n', '').replace('\t', '').replace('\s', '')
                         )).strip()
            
            if len(df) == 0:
                df.loc[len(df)] = [linha[0], elem.find_parent().find_parent().text, textappend]
            elif df['Texto semi-tratado'][len(df)-1] in textappend:
                df.loc[len(df)-1] = [linha[0], elem.find_parent().find_parent().text, textappend]
            else:
                df.loc[len(df)] = [linha[0], elem.find_parent().find_parent().text, textappend]
                
        bar.update(i*1000+page)
        
        time.sleep(2 + random.uniform(0, 3))
    i += 1

In [11]:
# Exportando para csv
df.to_csv(r'ementas.csv')

In [12]:
# Verificando número de ementas por assunto
df.groupby('Assunto').count()

,Texto original,Texto semi-tratado
Assunto,,
Direito Civil,19838,19838
Direito Penal,17574,17574
Direito Previdenciário,19347,19347
Direito Tributário,32271,32271
Direito do Trabalho,18307,18307
